# Results for RoBERTa when applying syn tranformation to both premise and hypothesis

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from IPython.display import display, HTML 
from lr.analysis.util import get_ts_from_results_folder 
from lr.analysis.util import get_rho_stats_from_result_list
from lr.stats.h_testing import get_ks_stats_from_p_values_compared_to_uniform_dist

## Get Results

In [2]:
all_accs = []
all_transformed_accs = []
all_paired_t_p_values = []
all_dev_plus_diff = []
all_time = []

m_name = "roberta_base"
folder = "mnli"

test_repetitions = 4
batchs = range(1, test_repetitions + 1)
batchs = [1,2,3,4,6]

for i in tqdm(batchs):
    test_accuracy  = get_ts_from_results_folder(path="results/{}/{}/syn_p_h/batch{}/".format(folder,m_name, i),
                                                 stat="test_accuracy")
    
    transformed_test_accuracy = get_ts_from_results_folder(path="results/{}/{}/syn_p_h/batch{}/".format(folder,m_name, i),
                                                           stat="transformed_test_accuracy")
    
    paired_t_p_value  = get_ts_from_results_folder(path="results/{}/{}/syn_p_h/batch{}/".format(folder,m_name, i),
                                                    stat="paired_t_p_value")
    
    diff  = get_ts_from_results_folder(path="results/{}/{}/syn_p_h/batch{}/".format(folder,m_name, i),
                                                    stat="dev_plus_accuracy_difference")
    
    t_time  = get_ts_from_results_folder(path="results/{}/{}/syn_p_h/batch{}/".format(folder,m_name,i),
                                                    stat="test_time")

    
    all_accs.append(test_accuracy)
    all_transformed_accs.append(transformed_test_accuracy)
    all_paired_t_p_values.append(paired_t_p_value)
    all_dev_plus_diff.append(diff)
    all_time.append(t_time)

100%|██████████| 5/5 [00:01<00:00,  3.85it/s]


In [3]:
pd.concat(all_accs,1)

,test_accuracy,test_accuracy,test_accuracy,test_accuracy,test_accuracy
0.0,0.827760,0.827760,0.596020,0.829999,0.000000
0.2,0.818242,0.836871,0.828880,0.328447,0.000000
0.4,0.831526,0.829338,0.318217,0.828065,0.832748
0.5,0.831068,0.661831,0.827607,0.328447,0.820227
0.6,0.353336,0.823586,0.827709,0.830814,0.827556
0.8,0.825571,0.724691,0.818344,0.328447,0.000000
1.0,0.797272,0.824095,0.828422,0.328447,0.000000


In [4]:
sfdfs

NameError: name 'sfdfs' is not defined

In [ ]:
total_time = pd.concat(all_time,1).sum().sum()
n_params = 125238531
print("Time for all experiments = {:.1f} hours".format(total_time))
print("Number of paramaters for RoBERTa = {}".format(n_params))

## Accuracy

In [ ]:
rhos, mean_acc, error_acc, _ = get_rho_stats_from_result_list(all_accs)

_, mean_acc_t, error_acc_t, _ = get_rho_stats_from_result_list(all_transformed_accs)

fig, ax = plt.subplots(figsize=(12,6))
ax.errorbar(rhos, mean_acc, yerr=error_acc, fmt='-o', label="original test data");
ax.errorbar(rhos, mean_acc_t, yerr=error_acc_t, fmt='-o', label="transformed test data");
ax.legend(loc="best");
ax.set_xlabel(r"$\rho$", fontsize=14);
ax.set_ylabel("accuracy", fontsize=14);
ax.set_title("RoBERTa accuracy\n\ndataset: MNLI\ntransformation: synonym substitution\ntest repetitions: {}\n".format(test_repetitions));
fig.tight_layout()
fig.savefig('figs/roberta_base_acc_mnli_syn_p_h.png', bbox_inches=None, pad_inches=0.5)

## P-values

In [ ]:
rhos, mean_p_values, error_p_values, min_p_values = get_rho_stats_from_result_list(all_paired_t_p_values)

alpha = 0.05
alpha_adj = alpha / test_repetitions

rejected_ids = []
remain_ids = []

for i,p in enumerate(min_p_values):
    if p < alpha_adj:
        rejected_ids.append(i)
    else:
        remain_ids.append(i)
        
rhos_rejected = rhos[rejected_ids]
rhos_remain = rhos[remain_ids]
y_rejected = mean_p_values[rejected_ids]
y_remain = mean_p_values[remain_ids]
error_rejected = error_p_values[rejected_ids]
error_remain = error_p_values[remain_ids]

title_msg = "RoBERTa p-values\n\ndataset:"
title_msg += "MNLI\ntransformation: synonym substitution\ntest repetitions: {}\n".format(test_repetitions)
title_msg += "significance level = {:.1%} \n".format(alpha)
title_msg += "adjusted significance level = {:.2%} \n".format(alpha_adj)


fig, ax = plt.subplots(figsize=(12,6))
ax.errorbar(rhos_rejected, y_rejected, yerr=error_rejected, fmt='o', linewidth=0.50, label="at least one p-value is smaller than {:.2%}".format(alpha_adj));
ax.errorbar(rhos_remain, y_remain, yerr=error_remain, fmt='o', linewidth=0.50, label="all p-values are greater than {:.2%}".format(alpha_adj));
ax.legend(loc="best");
ax.set_xlabel(r"$\rho$", fontsize=14);
ax.set_ylabel("p-value", fontsize=14);
ax.set_title(title_msg);
fig.tight_layout()
fig.tight_layout()
fig.savefig('figs/roberta_p_values_mnli_syn_p_h.png', bbox_inches=None, pad_inches=0.5)

## Accuracy difference

In [ ]:
rhos, diff, _,_ = get_rho_stats_from_result_list(all_dev_plus_diff)
_, test_acc, _,_ = get_rho_stats_from_result_list(all_accs)
_, test_acc_t, _,_ = get_rho_stats_from_result_list(all_transformed_accs)
test_diff = np.abs(test_acc - test_acc_t)

fig, ax = plt.subplots(figsize=(12,6))
ax.errorbar(rhos, diff, fmt='-o', label="validation");
ax.errorbar(rhos, test_diff, fmt='-o', label="test");

ax.legend(loc="best");
ax.set_xlabel(r"$\rho$", fontsize=14);
ax.set_ylabel("average accuracy difference", fontsize=14);
ax.set_title("RoBERTa accuracy difference\n\ndataset: MNLI\ntransformation: synonym substitution\ntest repetitions: {}\n".format(test_repetitions));
fig.tight_layout()
fig.savefig('figs/roberta_acc_diff_mnli_syn_p_h.png', bbox_inches=None, pad_inches=0.5)

## Selecting the best $\rho$

In [ ]:
id_min = np.argmin(diff)
min_rho = rhos[id_min]
min_rho_test_acc = test_acc[id_min]
min_rho_transformed_test_acc = test_acc_t[id_min]
test_accuracy_loss_pct = np.round(((min_rho_test_acc  - test_acc[0]) / test_acc[0]) * 100, 1)

analysis = {"dataset":"mnli",
            "model": "RoBERTa",
            "rho":min_rho,
            "test_accuracy_loss_pct": test_accuracy_loss_pct,
            "average_test_accuracy": min_rho_test_acc,
            "average_transformed_test_accuracy": min_rho_transformed_test_acc,
            "combined_accuracy": np.mean([min_rho_test_acc,min_rho_transformed_test_acc])}
analysis = pd.DataFrame(analysis, index=[0])
analysis